In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

2025-09-05 04:04:26.800046: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-05 04:04:26.953640: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
#Check GPU availability
print("GPU Available: ", tf.config.list_physical_devices('GPU'))
print("Built with CUDA: ", tf.test.is_built_with_cuda())

# Force GPU usage if available
if tf.config.list_physical_devices('GPU'):
    print("Using GPU for training")
    # Optional: Set memory growth to avoid GPU memory issues
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            tf.config.experimental.set_memory_growth(gpus[0], True)
        except RuntimeError as e:
            print(e)

GPU Available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Built with CUDA:  True
Using GPU for training


2025-09-05 04:04:30.059351: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-09-05 04:04:30.090838: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-09-05 04:04:30.093903: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [3]:
# Load and prepare data
df = pd.read_csv('../data/mucnuoc_gio_preprocess.csv')
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date').reset_index(drop=True)

# Setup features and target
features = ['q64']  # Use all features for input
target = 'q64'  # Target feature for prediction

In [4]:
# Normalize data
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

# Scale all features for X
scaled_data = scaler_X.fit_transform(df[features])

# Scale only target feature for y (important for inverse transform)
target_data = scaler_y.fit_transform(df[[target]])


In [10]:
# Prepare sequences
X, y = [], []
target_idx = features.index(target)
past_window = 180
future_window = 24

for i in range(len(scaled_data) - past_window - future_window + 1):
    # Input: past_window timesteps with all features
    X_window = scaled_data[i:i+past_window]
    
    # Output: future_window timesteps with only target feature
    # Use target_data (scaled separately) instead of scaled_data
    y_sequence = target_data[i+past_window:i+past_window+future_window].flatten()
    
    X.append(X_window)
    y.append(y_sequence)

X = np.array(X, dtype=np.float32)  # Shape: (samples, past_window, n_features)
y = np.array(y, dtype=np.float32)  # Shape: (samples, future_window)

print(f"📊 Data shape: X={X.shape}, y={y.shape}")


📊 Data shape: X=(47300, 180, 1), y=(47300, 24)


In [11]:
# Train/test split
split_idx = int(len(X) * 0.8)
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]

print(f"📊 Train shape: X_train={X_train.shape}, y_train={y_train.shape}")
print(f"📊 Test shape: X_test={X_test.shape}, y_test={y_test.shape}")


📊 Train shape: X_train=(37840, 180, 1), y_train=(37840, 24)
📊 Test shape: X_test=(9460, 180, 1), y_test=(9460, 24)


In [12]:
# Define parameter grid
param_grid = {
    'batch_size': [32, 64],
    'epochs': [20, 30],
    'learning_rate': [0.001, 0.01],
    'num_units': [32, 64],
    'dropout_rate': [0.2, 0.3]
}

In [13]:
def create_model(num_units, learning_rate, dropout_rate, input_shape, output_dim):
    """Create RNN model with given parameters"""
    model = Sequential([
        SimpleRNN(num_units, return_sequences=True, input_shape=input_shape),
        Dropout(dropout_rate),
        SimpleRNN(num_units // 2, return_sequences=False),
        Dropout(dropout_rate),
        Dense(64, activation='relu'),
        Dense(output_dim)
    ])
    
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    return model

In [14]:
# Grid search with proper model recreation
best_params = None
best_score = float('inf')
best_model = None

print("🔍 Starting Grid Search...")
total_combinations = len(list(ParameterGrid(param_grid)))
current_combination = 0

for params in ParameterGrid(param_grid):
    current_combination += 1
    print(f"\n📈 Testing combination {current_combination}/{total_combinations}: {params}")
    
    # Create new model for each parameter combination
    model = create_model(
        num_units=params['num_units'],
        learning_rate=params['learning_rate'],
        dropout_rate=params['dropout_rate'],
        input_shape=(past_window, len(features)),
        output_dim=future_window
    )
    
    # Early stopping to prevent overfitting
    early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    
    # Train model
    history = model.fit(
        X_train, y_train,
        epochs=params['epochs'],
        batch_size=params['batch_size'],
        validation_split=0.2,
        callbacks=[early_stop],
        verbose=0
    )
    
    # Evaluate on test set
    y_pred = model.predict(X_test, verbose=0)
    mse = mean_squared_error(y_test, y_pred)
    
    print(f"   MSE: {mse:.6f}")
    
    # Update best parameters
    if mse < best_score:
        best_score = mse
        best_params = params.copy()
        best_model = model
        print(f"   🎯 New best score!")

print(f"\n🏆 Best Parameters: {best_params}")
print(f"🏆 Best MSE: {best_score:.6f}")

🔍 Starting Grid Search...

📈 Testing combination 1/32: {'batch_size': 32, 'dropout_rate': 0.2, 'epochs': 20, 'learning_rate': 0.001, 'num_units': 32}
   MSE: 0.010866
   🎯 New best score!

📈 Testing combination 2/32: {'batch_size': 32, 'dropout_rate': 0.2, 'epochs': 20, 'learning_rate': 0.001, 'num_units': 64}
   MSE: 0.000234
   🎯 New best score!

📈 Testing combination 3/32: {'batch_size': 32, 'dropout_rate': 0.2, 'epochs': 20, 'learning_rate': 0.01, 'num_units': 32}
   MSE: 0.008107

📈 Testing combination 4/32: {'batch_size': 32, 'dropout_rate': 0.2, 'epochs': 20, 'learning_rate': 0.01, 'num_units': 64}
   MSE: 0.185479

📈 Testing combination 5/32: {'batch_size': 32, 'dropout_rate': 0.2, 'epochs': 30, 'learning_rate': 0.001, 'num_units': 32}
   MSE: 0.004122

📈 Testing combination 6/32: {'batch_size': 32, 'dropout_rate': 0.2, 'epochs': 30, 'learning_rate': 0.001, 'num_units': 64}
   MSE: 0.000371

📈 Testing combination 7/32: {'batch_size': 32, 'dropout_rate': 0.2, 'epochs': 30, 'lear

KeyboardInterrupt: 